# Parallel Chains

Similarly to how sequential chains connect chains in series, parallel chains connect chains in parallel. This is a useful abstraction when you want to independently process inputs with multiple different chains ("child chains), possibly asynchronously. The outputs of the child chains are merged together as a nested dictionary.

The word "parallel" here is to be interpreted as "independent" rather than "concurrent", and refers to the topology of how the chains are connected. Therefore, while the chains can be run concurrently, they are not run in parallel in the sense of being run on different threads or processes.

In the following examples, we wlll show:
- how we can use `ParallelChain` to take a list of chains and apply each independently to their inputs.
- how we can nest `ParallelChain`s inside `ParallelChain`s
- how we can use `ParallelChain` where the child chains take different inputs.
- how we can 

In [1]:
import pprint
import time

from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, ParallelChain

In [4]:
"""
In Jupyter notebooks, the default event loop policy is set to asyncio.get_event_loop_policy().ipython_kernel, 
which is different from the default event loop policy used by the Python interpreter. 
This can cause issues if we're running code that assumes the default event loop policy is being used.

To resolve this issue, we explicitly set the event loop policy in the Jupyter notebook to match the 
one used by the Python interpreter:
"""
import nest_asyncio

# Set the event loop policy to the default used by the Python interpreter
nest_asyncio.apply()

## Simple Example

In [5]:
llm = OpenAI(temperature=0.9)

input_variables=['product']

prompt_1 = PromptTemplate(
    input_variables=input_variables,
    template="What is a good name for a company that makes {product}?",
)
chain_1 = LLMChain(llm=llm, prompt=prompt_1)

prompt_2 = PromptTemplate(
    input_variables=input_variables,
    template="What is a good mission statement for a company that makes {product}?",
)
chain_2 = LLMChain(llm=llm, prompt=prompt_2)

prompt_3 = PromptTemplate(
    input_variables=input_variables,
    template="What is a good slogan for a company that makes {product}?",
)
chain_3 = LLMChain(llm=llm, prompt=prompt_3)

prompt_4 = PromptTemplate(
    input_variables=input_variables,
    template="What are some core values for a company that makes {product}?",
)
chain_4 = LLMChain(llm=llm, prompt=prompt_4)

The `ParallelChain`'s `concurrent` flag is set to `True` by default. When the `concurrent` flag is set to `True`, we can run the child chains concurrently. 

In [6]:
parallel_chain = ParallelChain(
    chains={
        'name': chain_1, 
        'mission': chain_2, 
        'slogan': chain_3,
        'values': chain_4
    },
    verbose=True,
    concurrent=True
)
s = time.perf_counter()
output = parallel_chain("colorful socks")
pprint.pprint(output)
print('\033[1m' + f"Concurrent executed in {time.perf_counter()-s:0.2f} seconds." + '\033[0m')



> Entering new ParallelChain chain...
Child chain for key="name" started.
Child chain for key="mission" started.
Child chain for key="slogan" started.
Child chain for key="values" started.
Child chain for key="slogan" finished after 2.11 seconds.Child chain for key="name" finished after 2.11 seconds.

Child chain for key="mission" finished after 3.13 seconds.
Child chain for key="values" finished after 3.69 seconds.

> Finished chain.
{'mission': {'text': '\n'
                     '\n'
                     'Our mission at [Company Name] is to bring joy and '
                     "individuality to every person's wardrobe by offering a "
                     'wide range of exceptional, vibrant, and comfortable '
                     'socks that turn any outfit into a fashionable '
                     'statement.'},
 'name': {'text': '\n\nRainbow Sock Shop.'},
 'product': 'colorful socks',
 'slogan': {'text': '\n\n"Live Life Colorfully with Our Socks!"'},
 'values': {'text': '\n'
     

Setting the `concurrent` flag to `False` would run the child chains serially.

In [7]:
parallel_chain.concurrent=False
s = time.perf_counter()
output = parallel_chain("colorful socks")
pprint.pprint(output)
print('\033[1m' + f"Serial executed in {time.perf_counter()-s:0.2f} seconds." + '\033[0m')



> Entering new ParallelChain chain...
Child chain for key="name" started.
Child chain for key="name" finished after 0.91 seconds.
Child chain for key="mission" started.
Child chain for key="mission" finished after 1.57 seconds.
Child chain for key="slogan" started.
Child chain for key="slogan" finished after 1.39 seconds.
Child chain for key="values" started.
Child chain for key="values" finished after 3.66 seconds.

> Finished chain.
{'mission': {'text': '\n'
                     '\n'
                     'Our mission is to provide our customers with expressive, '
                     'creative, and comfortable socks that make them feel '
                     'confident and happy every day.'},
 'name': {'text': '\n\nRainbow Socks Co.'},
 'product': 'colorful socks',
 'slogan': {'text': '\n\n"Socks with Style - Brighten Up Your Step!"'},
 'values': {'text': '\n'
                    '\n'
                    '1. Quality: Providing the highest quality of materials, '
                   

## Nesting `ParallelChain`s
It is possible to nest `ParallelChain`s inside one another. Continuing from the previous example, we nest a concurrent `ParallelChain` inside the previous serial `ParallelChain`.

In [8]:
prompt_5_1 = PromptTemplate(
    input_variables=input_variables,
    template="Which gift would go well with {product}?",
)
chain_5_1 = LLMChain(llm=llm, prompt=prompt_5_1)

prompt_5_2 = PromptTemplate(
    input_variables=input_variables,
    template="What gift would not go well with {product}?",
)
chain_5_2 = LLMChain(llm=llm, prompt=prompt_5_2)

chain_5 = ParallelChain(
    chains={'good_gift': chain_5_1, 'bad_gift': chain_5_2},
    verbose=True,
    concurrent=True
)

parallel_chain.chains.update({'gift': chain_5})

output = parallel_chain("colorful socks")
pprint.pprint(output)



> Entering new ParallelChain chain...
Child chain for key="name" started.
Child chain for key="name" finished after 0.76 seconds.
Child chain for key="mission" started.
Child chain for key="mission" finished after 1.56 seconds.
Child chain for key="slogan" started.
Child chain for key="slogan" finished after 0.61 seconds.
Child chain for key="values" started.
Child chain for key="values" finished after 5.58 seconds.
Child chain for key="gift" started.


> Entering new ParallelChain chain...
Child chain for key="good_gift" started.
Child chain for key="bad_gift" started.
Child chain for key="bad_gift" finished after 0.46 seconds.
Child chain for key="good_gift" finished after 0.70 seconds.

> Finished chain.
Child chain for key="gift" finished after 0.70 seconds.

> Finished chain.
{'gift': {'bad_gift': {'text': '\n\nA vacuum cleaner.'},
          'good_gift': {'text': '\n'
                                '\n'
                                'A matching pair of slippers would go well 

We can now make the outer `ParallelChain` execute concurrently again by setting `parallel_chain.concurrent=True`. Executing this nested chain will result in executing a concurrent `ParallelChain` inside another concurrent `ParallelChain`.

In [9]:
parallel_chain.concurrent=True
output = parallel_chain("colorful socks")
pprint.pprint(output)



> Entering new ParallelChain chain...
Child chain for key="name" started.
Child chain for key="mission" started.
Child chain for key="slogan" started.
Child chain for key="values" started.
Child chain for key="gift" started.


> Entering new ParallelChain chain...
Child chain for key="good_gift" started.
Child chain for key="bad_gift" started.
Child chain for key="bad_gift" finished after 0.73 seconds.
Child chain for key="name" finished after 0.82 seconds.
Child chain for key="good_gift" finished after 0.87 seconds.

> Finished chain.
Child chain for key="gift" finished after 0.88 seconds.
Child chain for key="slogan" finished after 0.92 seconds.
Child chain for key="mission" finished after 1.77 seconds.
Child chain for key="values" finished after 4.02 seconds.

> Finished chain.
{'gift': {'bad_gift': {'text': '\n\nA book.'},
          'good_gift': {'text': '\n'
                                '\n'
                                'A graphic T-shirt or hoodie would be a great '
     

## Different inputs in the child chains
So far, all our examples have shown all child chains taking the same input. But it is also possible for different child chains to take in different inputs as well. Here is an example where different child chains take different inputs.

In [12]:
llm = OpenAI(temperature=0.9)

prompt_1 = PromptTemplate(
    input_variables=['attribute_1', 'product'],
    template="What is a good {attribute_1} for a company that makes {product}?",
)
chain_1 = LLMChain(llm=llm, prompt=prompt_1)

prompt_2 = PromptTemplate(
    input_variables=['attribute_2', 'product'],
    template="What is a good {attribute_2} for a company that makes {product}?",
)
chain_2 = LLMChain(llm=llm, prompt=prompt_2)

prompt_3 = PromptTemplate(
    input_variables=['attribute_2', 'food'],
    template="What is a good {attribute_2} for a restaurant that makes {food}?",
)
chain_3 = LLMChain(llm=llm, prompt=prompt_3)


parallel_chain = ParallelChain(
    chains={
        'name_company': chain_1, 
        'slogan_company': chain_2,
        'slogan_restaurant': chain_3,
    },
    verbose=True,
    concurrent=True
)
s = time.perf_counter()
output = parallel_chain({
    "attribute_1": "name", 
    "attribute_2": "slogan", 
    "product": "colorful socks",
    "food": "pasta",
})
pprint.pprint(output)
print('\033[1m' + f"Concurrent executed in {time.perf_counter()-s:0.2f} seconds." + '\033[0m')



> Entering new ParallelChain chain...
Child chain for key="name_company" started.
Child chain for key="slogan_company" started.
Child chain for key="slogan_restaurant" started.
Child chain for key="name_company" finished after 0.74 seconds.
Child chain for key="slogan_restaurant" finished after 0.97 seconds.
Child chain for key="slogan_company" finished after 1.80 seconds.

> Finished chain.
{'attribute_1': 'name',
 'attribute_2': 'slogan',
 'food': 'pasta',
 'name_company': {'text': '\n\nBrightSox.'},
 'product': 'colorful socks',
 'slogan_company': {'text': '\n\n"Brighten Your Step with Colorful Socks!"'},
 'slogan_restaurant': {'text': '\n'
                               '\n'
                               '"Taste the Italian Tradition with Our Homemade '
                               'Pasta!"'}}
Concurrent executed in 1.80 seconds.
